In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import minmax_scale, scale
from tqdm import tqdm
import time
import implicit

c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\impicit_mix_als_bpr_hyper_v2_pred"
target_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\impicit_mix_als_bpr_hyper_v2_target"

In [3]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")
    
    return mrr_score / len(y_true)

In [4]:
%%time

list_coord = []
list_coord_val = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(10000), size=2500, replace=False)
val_id.sort()

with open(train_path) as f:
    lines = f.readlines()
    idx = 0
    idx_val = 0
    for line in lines:
        tracks = line.strip().split(' ')
        if len(tracks) == 0:
            print(0)
        if len(tracks) == 1:
            print(1)
        for i, track in enumerate(tracks):
            if idx in val_id and i+1 == len(tracks):
                target_val.append(track)
                idx_val += 1
                continue
            if idx in val_id:
                list_coord_val.append((idx_val, track))
            list_coord.append((idx, track))
        idx += 1
        if idx >= 10000:
            break

CPU times: total: 4.91 s
Wall time: 4.89 s


In [5]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)
#item_user = csr_matrix((ones, (arr_cols, arr_rows)), dtype=int)

arr_rows_val = np.array([i[0] for i in list_coord_val])
arr_cols_val = np.array([i[1] for i in list_coord_val]).astype(int)
ones_val = np.ones(arr_rows_val.shape[0])
user_item_val = csr_matrix((ones_val, (arr_rows_val, arr_cols_val)), dtype=int)

CPU times: total: 438 ms
Wall time: 432 ms


# baseline

In [6]:
model_als = implicit.als.AlternatingLeastSquares(factors=200, 
                                                 regularization=10,
                                                 alpha=40,
                                                 use_native=True,
                                                 use_cg=False,
                                                 iterations=15,
                                                 random_state=42)
print('model_als.fit:')
model_als.fit(user_item)

start_time = time.time()
userid = val_id
rec_als = model_als.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

result = [' '.join(map(str, i)) + '\n' for i in rec_als[0]]

    
with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

model_als.fit:


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [14:52<00:00, 59.53s/it]




--- 9.55253553390503 seconds model_als ---
MRR@100 = 0.0197


0.019655344382911492

In [7]:
print('model_als.fit:')
model_als.fit(user_item)

start_time = time.time()
userid = val_id
rec_als = model_als.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

result = [' '.join(map(str, i)) + '\n' for i in rec_als[0]]

    
with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

model_als.fit:


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [14:31<00:00, 58.11s/it]




--- 6.573267221450806 seconds model_als ---
MRR@100 = 0.0202


0.020163782089434375

In [8]:
print('model_als.fit:')
model_als.fit(user_item)

start_time = time.time()
userid = val_id
rec_als = model_als.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

result = [' '.join(map(str, i)) + '\n' for i in rec_als[0]]

    
with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

model_als.fit:


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [14:32<00:00, 58.15s/it]




--- 7.718630313873291 seconds model_als ---
MRR@100 = 0.0204


0.020375250129893302

In [9]:
print('model_als.fit:')
model_als.fit(user_item)

start_time = time.time()
userid = val_id
rec_als = model_als.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

result = [' '.join(map(str, i)) + '\n' for i in rec_als[0]]

    
with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

model_als.fit:


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [14:47<00:00, 59.16s/it]




--- 7.337726354598999 seconds model_als ---
MRR@100 = 0.0205


0.02048628316120411

In [10]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                     learning_rate=0.075,
                                                     regularization=0.0025,
                                                     iterations=950,
                                                     verify_negative_samples=True,
                                                     random_state=42)   
print('model_bpr.fit:')
model_bpr.fit(user_item)

start_time = time.time()
userid = val_id
rec_bpr = model_bpr.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
print("\n\n--- %s seconds model_bpr ---" % (time.time() - start_time))

result = [' '.join(map(str, i)) + '\n' for i in rec_bpr[0]]

    
with open(pred_path, 'w') as f:
    f.writelines(result)

with open(target_path, 'w') as f:
    for i in target_val:
        f.write(f'{i}\n')

calc_score(target_path, pred_path)

model_bpr.fit:


100%|███████████████████████████████████████████████| 950/950 [04:14<00:00,  3.73it/s, train_auc=99.99%, skipped=0.78%]




--- 11.3808913230896 seconds model_bpr ---
MRR@100 = 0.0208


0.020833969176156264

# ALS: 0.0205
# BPR: 0.0208

## Tests

In [11]:
def N_opt(model_als, model_bpr, N=100, q_bpr=1):
    
    N_als = N
    N_bpr = int(N * q_bpr)
    
    userid = val_id
        
    start_time = time.time()
    rec_als = model_als.recommend(userid, user_item_val, N=N_als, filter_already_liked_items=True)
    print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

    start_time = time.time()
    rec_bpr = model_bpr.recommend(userid, user_item_val, N=N_bpr, filter_already_liked_items=True)
    print("\n--- %s seconds model_bpr ---" % (time.time() - start_time))
    
    return rec_als, rec_bpr, N_als, N_bpr

In [12]:
def optimize(rec_als,
             rec_bpr,
             N_als,
             N_bpr,
             coef_bpr=1,
             norm_all=False,
             preproc=None):
    
    if not isinstance(norm_all, bool) or preproc not in ['minmax', 'standart', None]:
        raise ValueError('wrong params')
        
    if preproc is None:
        als_score = rec_als[1]
        bpr_score = rec_bpr[1]
    else:
        if norm_all:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_als))
                bpr_score = minmax_scale(rec_bpr[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_bpr))
            if preproc == 'standart':
                als_score = scale(rec_als[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_als))
                bpr_score = scale(rec_bpr[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_bpr))    
        else:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1], feature_range=(0, 1), axis=1)
                bpr_score = minmax_scale(rec_bpr[1], feature_range=(0, 1), axis=1)   
            if preproc == 'standart':
                als_score = scale(rec_als[1], with_mean=True, with_std=True, axis=1)
                bpr_score = scale(rec_bpr[1], with_mean=True, with_std=True, axis=1)     

    result = []           
    for i in tqdm(range(rec_als[0].shape[0])):
        als_df = pd.DataFrame({'trak_id': rec_als[0][i], 'als_score': als_score[i]})
        bpr_df = pd.DataFrame({'trak_id': rec_bpr[0][i], 'bpr_score': bpr_score[i] * coef_bpr})
        df = pd.merge(als_df, bpr_df, how="outer", on='trak_id', sort=False)
        df['als_prior'] = df.als_score.where(~df.als_score.isna(), df.bpr_score)
        df['total_score'] = np.where((df.als_score.isna()) | (df.bpr_score.isna()), df.als_prior, 
                                     (df.als_score + df.bpr_score) / 2)
        final_rec = df.sort_values(by='total_score', ascending=False).trak_id.values[:100]
        result.append(' '.join(map(str, final_rec)) + '\n')
        
        
    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    mrr = calc_score(target_path, pred_path)

    return mrr

## Final best

baseline 0.0205

mrr	0.024815

N 542

q_bpr 0.015726

norm_all True

preproc minmax

coef_bpr 0.438263

In [13]:
rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=542, q_bpr=0.015726)
optimize(rec_als,
           rec_bpr,
           N_als,
           N_bpr,
           coef_bpr=0.438263,
           norm_all=True,
           preproc='minmax')



--- 6.060504674911499 seconds model_als ---

--- 11.417316913604736 seconds model_bpr ---


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 394.63it/s]


MRR@100 = 0.0197


0.01971195846063265

In [14]:
ls = []

In [15]:
ls = []
for N in [100, 250, 500, 750, 1000, 1500, 2000]:
    for q_bpr in [0.01, 0.1, 0.5, 1, 2, 10, 100]:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for norm_all in [True, False]:
            for preproc in [None, 'minmax', 'standart']:
                for coef_bpr in [0.005, 0.01, 0.1, 0.5, 1, 10, 100]:
                    print(f'N: {N}, q_bpr: {q_bpr}, norm_all: {norm_all}, preproc: {preproc}, coef_bpr: {coef_bpr}')
                    mrr = optimize(rec_als,
                                   rec_bpr,
                                   N_als,
                                   N_bpr,
                                   coef_bpr=coef_bpr,
                                   norm_all=norm_all,
                                   preproc=preproc)
                    ls.append((mrr, N, q_bpr, norm_all, preproc, coef_bpr))



--- 6.994697332382202 seconds model_als ---

--- 15.063284158706665 seconds model_bpr ---
N: 100, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.005


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 421.81it/s]


MRR@100 = 0.0187
N: 100, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.01


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 424.38it/s]


MRR@100 = 0.0187
N: 100, q_bpr: 0.01, norm_all: True, preproc: None, coef_bpr: 0.1


...

MRR@100 = 0.0190
N: 2000, q_bpr: 100, norm_all: False, preproc: standart, coef_bpr: 10


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [01:38<00:00, 25.33it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data an

MRR@100 = 0.0028
N: 2000, q_bpr: 100, norm_all: False, preproc: standart, coef_bpr: 100


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [01:37<00:00, 25.63it/s]


MRR@100 = 0.0018


In [16]:
data = pd.DataFrame(ls, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
1572,0.023579,1500,0.50,True,standart,1.0
1866,0.023562,2000,0.50,True,standart,1.0
1908,0.023507,2000,1.00,True,standart,1.0
1383,0.023322,1000,2.00,False,standart,1.0
1635,0.023317,1500,1.00,False,standart,1.0
1677,0.023249,1500,2.00,False,standart,1.0
1929,0.023236,2000,1.00,False,standart,1.0
1047,0.023075,750,1.00,False,standart,1.0
1586,0.023069,1500,0.50,False,minmax,1.0
1648,0.023067,1500,2.00,True,minmax,0.5


In [18]:
ls_fine = []
for N in [1250, 1500, 1750]:
    for q_bpr in [0.25, 0.5, 0.75]:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for coef_bpr in [0.75, 1, 1.25]:
            print(f'N: {N}, q_bpr: {q_bpr}, norm_all: True, preproc: standart, coef_bpr: {coef_bpr}')
            mrr = optimize(rec_als,
                           rec_bpr,
                           N_als,
                           N_bpr,
                           coef_bpr=coef_bpr,
                           norm_all=True,
                           preproc='standart')
            ls_fine.append((mrr, N, q_bpr, True, 'standart', coef_bpr))



--- 5.904430389404297 seconds model_als ---


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(



--- 12.977040767669678 seconds model_bpr ---
N: 1250, q_bpr: 0.25, norm_all: True, preproc: standart, coef_bpr: 0.75


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:07<00:00, 347.79it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


MRR@100 = 0.0225
N: 1250, q_bpr: 0.25, norm_all: True, preproc: standart, coef_bpr: 1


...

MRR@100 = 0.0230
N: 1750, q_bpr: 0.75, norm_all: True, preproc: standart, coef_bpr: 1


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:08<00:00, 297.38it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(


MRR@100 = 0.0236
N: 1750, q_bpr: 0.75, norm_all: True, preproc: standart, coef_bpr: 1.25


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:08<00:00, 299.76it/s]


MRR@100 = 0.0226


In [19]:
data_fine = pd.DataFrame(ls_fine, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data_fine.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
22,0.023785,1750,0.50,True,standart,1.00
13,0.023579,1500,0.50,True,standart,1.00
25,0.023561,1750,0.75,True,standart,1.00
4,0.023287,1250,0.50,True,standart,1.00
16,0.023157,1500,0.75,True,standart,1.00
19,0.023153,1750,0.25,True,standart,1.00
7,0.023134,1250,0.75,True,standart,1.00
21,0.023052,1750,0.50,True,standart,0.75
3,0.023046,1250,0.50,True,standart,0.75
12,0.023019,1500,0.50,True,standart,0.75


In [21]:
ls_rand = []

N_arr = np.linspace(1550, 1850, num=5, dtype=int)
q_bpr_arr = np.linspace(0.35, 0.65, num=5)
coef_bpr_arr = np.linspace(0.85, 1.15, num=5)

for N in N_arr:
    for q_bpr in q_bpr_arr:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for coef_bpr in coef_bpr_arr:
            print(f'N: {N}, q_bpr: {q_bpr}, norm_all: True, preproc: standart, coef_bpr: {coef_bpr}')
            mrr = optimize(rec_als,
                           rec_bpr,
                           N_als,
                           N_bpr,
                           coef_bpr=coef_bpr,
                           norm_all=True,
                           preproc='standart')
            ls_rand.append((mrr, N, q_bpr, True, 'standart', coef_bpr))



--- 8.613096237182617 seconds model_als ---


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(



--- 16.427449941635132 seconds model_bpr ---
N: 1550, q_bpr: 0.35, norm_all: True, preproc: standart, coef_bpr: 0.85


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:08<00:00, 302.10it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(


MRR@100 = 0.0233
N: 1550, q_bpr: 0.35, norm_all: True, preproc: standart, coef_bpr: 0.9249999999999999


...

MRR@100 = 0.0238
N: 1850, q_bpr: 0.65, norm_all: True, preproc: standart, coef_bpr: 1.075


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:08<00:00, 305.45it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(


MRR@100 = 0.0236
N: 1850, q_bpr: 0.65, norm_all: True, preproc: standart, coef_bpr: 1.15


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:07<00:00, 319.82it/s]


MRR@100 = 0.0236


In [22]:
data_rand = pd.DataFrame(ls_rand, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data_rand.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
31,0.023960,1625,0.425,True,standart,0.925
46,0.023885,1625,0.650,True,standart,0.925
6,0.023885,1550,0.425,True,standart,0.925
16,0.023868,1550,0.575,True,standart,0.925
82,0.023866,1775,0.425,True,standart,1.000
36,0.023865,1625,0.500,True,standart,0.925
41,0.023863,1625,0.575,True,standart,0.925
57,0.023863,1700,0.425,True,standart,1.000
80,0.023860,1775,0.425,True,standart,0.850
85,0.023831,1775,0.500,True,standart,0.850


In [23]:
model_als = implicit.als.AlternatingLeastSquares(factors=200, 
                                                 regularization=10,
                                                 alpha=40,
                                                 use_native=True,
                                                 use_cg=False,
                                                 iterations=15,
                                                 random_state=42)
print('model_als.fit:')
model_als.fit(user_item)

model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                     learning_rate=0.075,
                                                     regularization=0.0025,
                                                     iterations=950,
                                                     verify_negative_samples=True,
                                                     random_state=42)   
print('model_bpr.fit:')
model_bpr.fit(user_item)

model_als.fit:


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [14:30<00:00, 58.05s/it]


model_bpr.fit:


100%|███████████████████████████████████████████████| 950/950 [04:20<00:00,  3.65it/s, train_auc=99.99%, skipped=0.78%]


In [24]:
rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=1625, q_bpr=0.425)
optimize(rec_als,
           rec_bpr,
           N_als,
           N_bpr,
           coef_bpr=0.925,
           norm_all=True,
           preproc='standart')



--- 7.299200773239136 seconds model_als ---


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(



--- 12.593859672546387 seconds model_bpr ---


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:07<00:00, 348.70it/s]

MRR@100 = 0.0234


0.023423493043154362

In [25]:
rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=542, q_bpr=0.015726)
optimize(rec_als,
           rec_bpr,
           N_als,
           N_bpr,
           coef_bpr=0.438263,
           norm_all=True,
           preproc='minmax')



--- 6.809091329574585 seconds model_als ---

--- 13.356370210647583 seconds model_bpr ---


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 429.64it/s]


MRR@100 = 0.0192


0.01917300059406546

In [26]:
ls2 = []
for N in [500, 750, 1000, 1500]:
    for q_bpr in [0.01, 0.1, 0.5, 1]:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for preproc in ['minmax', 'standart']:
            for coef_bpr in [0.1, 0.5, 1]:
                print(f'N: {N}, q_bpr: {q_bpr}, norm_all: True, preproc: {preproc}, coef_bpr: {coef_bpr}')
                mrr = optimize(rec_als,
                               rec_bpr,
                               N_als,
                               N_bpr,
                               coef_bpr=coef_bpr,
                               norm_all=True,
                               preproc=preproc)
                ls2.append((mrr, N, q_bpr, True, preproc, coef_bpr))



--- 6.859510898590088 seconds model_als ---

--- 13.046378135681152 seconds model_bpr ---
N: 500, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 0.1


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 439.51it/s]


MRR@100 = 0.0155
N: 500, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 0.5


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:05<00:00, 430.50it/s]


MRR@100 = 0.0200
N: 500, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 1


...

MRR@100 = 0.0185
N: 1500, q_bpr: 1, norm_all: True, preproc: standart, coef_bpr: 0.5


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:07<00:00, 349.42it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(


MRR@100 = 0.0222
N: 1500, q_bpr: 1, norm_all: True, preproc: standart, coef_bpr: 1


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:07<00:00, 345.61it/s]

MRR@100 = 0.0232


In [27]:
data2 = pd.DataFrame(ls2, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data2.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
89,0.023665,1500,0.50,True,standart,1.0
74,0.023551,1500,0.01,True,minmax,1.0
50,0.023404,1000,0.01,True,minmax,1.0
95,0.023248,1500,1.00,True,standart,1.0
85,0.023112,1500,0.50,True,minmax,0.5
80,0.022965,1500,0.10,True,minmax,1.0
26,0.022923,750,0.01,True,minmax,1.0
91,0.022866,1500,1.00,True,minmax,0.5
65,0.022835,1000,0.50,True,standart,1.0
71,0.022717,1000,1.00,True,standart,1.0


In [28]:
model_als = implicit.als.AlternatingLeastSquares(factors=200, 
                                                 regularization=10,
                                                 alpha=10,
                                                 use_native=True,
                                                 use_cg=False,
                                                 iterations=15,
                                                 random_state=42)
print('model_als.fit:')
model_als.fit(user_item)

model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                     learning_rate=0.1,
                                                     regularization=0.01,
                                                     iterations=550,
                                                     verify_negative_samples=True,
                                                     random_state=42)   
print('model_als.fit:')
model_bpr.fit(user_item)

rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=542, q_bpr=0.015726)

optimize(rec_als,
           rec_bpr,
           N_als,
           N_bpr,
           coef_bpr=0.438263,
           norm_all=True,
           preproc='minmax')

model_als.fit:


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [14:42<00:00, 58.85s/it]


model_als.fit:


100%|███████████████████████████████████████████████| 550/550 [02:57<00:00,  3.09it/s, train_auc=99.73%, skipped=0.77%]




--- 8.203802108764648 seconds model_als ---

--- 13.93753981590271 seconds model_bpr ---


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:06<00:00, 390.01it/s]

MRR@100 = 0.0221


0.022139445099934197